In [127]:
# notebooks/Project_3a.ipynb

# Import system references
import sys
import os

# Ensure project_root is in the system path
current_dir = os.getcwd()
project_root = os.path.abspath(os.path.join(current_dir, '..'))
if project_root not in sys.path:
    sys.path.append(project_root)
    
#print("Project root added to sys.path:", project_root in sys.path)
#print(sys.path)



In [128]:
#setup Logging
import logging

# Configure logging
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s',
    handlers=[
        logging.StreamHandler()
    ]
)

logger = logging.getLogger(__name__)

# Configure logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')



In [129]:
# Import necessary libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from datetime import datetime
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

import torch
import torch.nn as nn
import torch.optim as optim

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.optimizers import Adam
from sklearn.preprocessing import MinMaxScaler

# Import the fetch_stock_data function
from scripts import fetch_stock_data, transform_stock_data_to_delta, transform_with_history
from scripts import prepare_data_for_training, create_time_series_windows, lstm_model, prepare_sliding_window_data

In [130]:
# Ask the user for a ticker symbol
ticker = input("Please enter a stock ticker symbol (e.g., 'AAPL', 'SPY', 'QQQ', etc.): ").upper()

# Define start and end dates
start_date = '2021-01-01'

# If end_date is not provided, you can set it to today's date
end_date = datetime.today().strftime('%Y-%m-%d')  # Default to today's date if not specified

# Print the ticker and date range to confirm
print(f"Fetching data for {ticker} from {start_date} to {end_date}")

Fetching data for VTI from 2021-01-01 to 2024-10-28


In [131]:
# Fetch stock data
stock_data_df = fetch_stock_data(ticker, start_date, end_date)
#stock_data_df = fetch_stock_data(ticker, start_date)

# Check if data is fetched successfully
if not stock_data_df.empty:
    # Display the first few rows
    display(stock_data_df.head())
else:
    print("No data to display.")

2024-10-28 14:34:43,777 - INFO - End date provided: 2024-10-28
2024-10-28 14:34:43,779 - INFO - Fetching data for ticker: VTI from 2021-01-01 to 2024-10-28
2024-10-28 14:34:44,308 - INFO - Resetting index to make 'Date' a column.
2024-10-28 14:34:44,310 - INFO - Flattening multi-level column names (removing ticker symbol).
2024-10-28 14:34:44,310 - INFO - Successfully fetched and simplified data for ticker 'VTI'.


Price,Date,Adj Close,Close,High,Low,Open,Volume
0,2021-01-04 00:00:00+00:00,181.573761,191.869995,195.429993,189.759995,195.389999,8825800
1,2021-01-05 00:00:00+00:00,183.040573,193.419998,193.949997,191.419998,191.490005,3894000
2,2021-01-06 00:00:00+00:00,184.687225,195.160004,196.889999,192.460007,192.500000,6175700
3,2021-01-07 00:00:00+00:00,187.649246,198.289993,198.619995,196.399994,196.419998,4830900
4,2021-01-08 00:00:00+00:00,188.557709,199.250000,199.460007,196.979996,199.139999,7538700


In [132]:
# Transform Stock Data to Deltas
columns_to_exclude = ['Adj Close']  # Drop 'Adj Close'
columns_to_keep = []  # Keep 'Volume' but exclude from delta calculation
columns_to_calculate = ['Open', 'High', 'Low', 'Close', 'Volume']  # Calculate deltas for 'Open' and 'Close'

transformed_data_df = transform_stock_data_to_delta(
    stock_data_df, 
    columns_to_exclude=columns_to_exclude, 
    columns_to_calculate=columns_to_calculate, 
    columns_to_keep=columns_to_keep
)

#transformed_data_df = transform_stock_data_to_delta(stock_data_df)
#transformed_data_df = transform_stock_data_to_delta(stock_data_df, exclude=['Volume'])

# Display Transformed Data
if not transformed_data_df.empty:
    display(transformed_data_df.head())
else:
    print("No transformed data to display.")


2024-10-28 14:34:44,333 - INFO - Starting transformation of stock data to deltas.
2024-10-28 14:34:44,334 - INFO - Dropping columns: ['Adj Close']
2024-10-28 14:34:44,336 - INFO - Calculating deltas for specified columns: ['Open', 'High', 'Low', 'Close', 'Volume']
2024-10-28 14:34:44,337 - INFO - Transforming column: Open
2024-10-28 14:34:44,339 - INFO - Transforming column: High
2024-10-28 14:34:44,342 - INFO - Transforming column: Low
2024-10-28 14:34:44,345 - INFO - Transforming column: Close
2024-10-28 14:34:44,348 - INFO - Transforming column: Volume
2024-10-28 14:34:44,350 - INFO - Dropping the first row with NaN values after delta calculation.
2024-10-28 14:34:44,352 - INFO - Successfully transformed stock data to deltas.


Price,Date,Close,High,Low,Open,Volume,Open_delta,High_delta,Low_delta,Close_delta,Volume_delta
1,2021-01-05 00:00:00+00:00,193.419998,193.949997,191.419998,191.490005,3894000,-0.0200,-0.0076,0.0087,0.0081,-0.5588
2,2021-01-06 00:00:00+00:00,195.160004,196.889999,192.460007,192.500000,6175700,0.0053,0.0152,0.0054,0.0090,0.5860
3,2021-01-07 00:00:00+00:00,198.289993,198.619995,196.399994,196.419998,4830900,0.0204,0.0088,0.0205,0.0160,-0.2178
4,2021-01-08 00:00:00+00:00,199.250000,199.460007,196.979996,199.139999,7538700,0.0138,0.0042,0.0030,0.0048,0.5605
5,2021-01-11 00:00:00+00:00,198.059998,199.059998,197.220001,197.419998,3858600,-0.0086,-0.0020,0.0012,-0.0060,-0.4882


In [133]:
# Prepare the data by removing non-delta columns
delta_only_df = prepare_sliding_window_data(transformed_data_df)

# Display the first few rows of the resulting DataFrame
display(delta_only_df.head())


Price,Open_delta,High_delta,Low_delta,Close_delta,Volume_delta
1,-0.0200,-0.0076,0.0087,0.0081,-0.5588
2,0.0053,0.0152,0.0054,0.0090,0.5860
3,0.0204,0.0088,0.0205,0.0160,-0.2178
4,0.0138,0.0042,0.0030,0.0048,0.5605
5,-0.0086,-0.0020,0.0012,-0.0060,-0.4882


In [134]:
# Define parameters
n_timesteps = 100  # Number of timesteps (sequence length)

# Step 1: Create the time series windows (X and y)
X, y = create_time_series_windows(delta_only_df, 'Close_delta', n_timesteps)

# Display the first few occurrences of the X and y arrays
print("First 5 entries of X:\n", X[:5])  # Display first 5 rows
print("First 5 entries of y:\n", y[:5])  # Display first 5 target values

# Step 2: Split the data into training and testing sets (80/20 split)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, shuffle=True)

# Display the shapes of the training and testing sets
print("X_train shape:", X_train.shape)
print("X_test shape:", X_test.shape)
print("y_train shape:", y_train.shape)
print("y_test shape:", y_test.shape)


First 5 entries of X:
 [[[-2.000e-02 -7.600e-03  8.700e-03  8.100e-03 -5.588e-01]
  [ 5.300e-03  1.520e-02  5.400e-03  9.000e-03  5.860e-01]
  [ 2.040e-02  8.800e-03  2.050e-02  1.600e-02 -2.178e-01]
  ...
  [ 7.400e-03  1.500e-03  1.300e-03 -2.800e-03  2.877e-01]
  [-3.900e-03 -2.100e-03  1.300e-03  3.600e-03 -1.196e-01]
  [ 4.700e-03  3.100e-03  5.100e-03  1.800e-03 -1.967e-01]]

 [[ 5.300e-03  1.520e-02  5.400e-03  9.000e-03  5.860e-01]
  [ 2.040e-02  8.800e-03  2.050e-02  1.600e-02 -2.178e-01]
  [ 1.380e-02  4.200e-03  3.000e-03  4.800e-03  5.605e-01]
  ...
  [-3.900e-03 -2.100e-03  1.300e-03  3.600e-03 -1.196e-01]
  [ 4.700e-03  3.100e-03  5.100e-03  1.800e-03 -1.967e-01]
  [ 2.800e-03  2.200e-03  2.000e-03  1.700e-03  7.100e-03]]

 [[ 2.040e-02  8.800e-03  2.050e-02  1.600e-02 -2.178e-01]
  [ 1.380e-02  4.200e-03  3.000e-03  4.800e-03  5.605e-01]
  [-8.600e-03 -2.000e-03  1.200e-03 -6.000e-03 -4.882e-01]
  ...
  [ 4.700e-03  3.100e-03  5.100e-03  1.800e-03 -1.967e-01]
  [ 2.800e-

In [135]:
# Initialize the scaler for X and y values
scaler_X = MinMaxScaler()
scaler_y = MinMaxScaler()

# Reshape X data to 2D for scaling, keeping the last dimension as features
n_samples_train = X_train.shape[0]
n_samples_test = X_test.shape[0]

X_train_reshaped = X_train.reshape(-1, X_train.shape[-1])  # Reshape to 2D: [samples * timesteps, features]
X_test_reshaped = X_test.reshape(-1, X_test.shape[-1])

# Apply scaling to X features (fit on X_train, transform both X_train and X_test)
X_train_scaled = scaler_X.fit_transform(X_train_reshaped).reshape(n_samples_train, X_train.shape[1], X_train.shape[2])
X_test_scaled = scaler_X.transform(X_test_reshaped).reshape(n_samples_test, X_test.shape[1], X_test.shape[2])

# Reshape y values to 2D (required by MinMaxScaler)
y_train = y_train.reshape(-1, 1)
y_test = y_test.reshape(-1, 1)

# Apply scaling to y values
y_train_scaled = scaler_y.fit_transform(y_train)
y_test_scaled = scaler_y.transform(y_test)



In [136]:
# Define parameters
#n_timesteps = 25
n_features = X_train_scaled.shape[2]

# Define the number of layers and units per layer
num_layers = 3
units_per_layer = [50, 100, 50]  # 3 layers with 50, 100, and 50 units, respectively

# Call the LSTM model
y_pred_scaled, model = lstm_model(X_train_scaled, y_train_scaled, X_test_scaled, y_test_scaled, n_timesteps, n_features, num_layers, units_per_layer, 
                         learning_rate=0.00001, epochs=70, batch_size=28)
# Output the predictions
#print("Predictions from LSTM model:", y_pred_scaled)



Epoch 1/70


c:\Users\mark.storer\Documents\Data\AI\Repositories\Project_3a\venv\Lib\site-packages\keras\src\layers\rnn\rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


25/25 - 9s - 352ms/step - loss: 0.1982 - val_loss: 0.1854
Epoch 2/70
25/25 - 3s - 113ms/step - loss: 0.1889 - val_loss: 0.1764
Epoch 3/70
25/25 - 3s - 118ms/step - loss: 0.1798 - val_loss: 0.1676
Epoch 4/70
25/25 - 3s - 124ms/step - loss: 0.1711 - val_loss: 0.1591
Epoch 5/70
25/25 - 3s - 127ms/step - loss: 0.1624 - val_loss: 0.1508
Epoch 6/70
25/25 - 3s - 114ms/step - loss: 0.1543 - val_loss: 0.1429
Epoch 7/70
25/25 - 3s - 116ms/step - loss: 0.1462 - val_loss: 0.1349
Epoch 8/70
25/25 - 3s - 116ms/step - loss: 0.1378 - val_loss: 0.1265
Epoch 9/70
25/25 - 3s - 114ms/step - loss: 0.1292 - val_loss: 0.1179
Epoch 10/70
25/25 - 3s - 121ms/step - loss: 0.1203 - val_loss: 0.1092
Epoch 11/70
25/25 - 3s - 118ms/step - loss: 0.1112 - val_loss: 0.1001
Epoch 12/70
25/25 - 3s - 120ms/step - loss: 0.1017 - val_loss: 0.0907
Epoch 13/70
25/25 - 3s - 121ms/step - loss: 0.0919 - val_loss: 0.0809
Epoch 14/70
25/25 - 3s - 115ms/step - loss: 0.0816 - val_loss: 0.0706
Epoch 15/70
25/25 - 3s - 127ms/step - lo

In [137]:
# Step 1: Inverse transform the predicted and actual y values to the original scale
y_pred_original = scaler_y.inverse_transform(y_pred_scaled)
y_test_original = scaler_y.inverse_transform(y_test_scaled)

# Display the first few occurrences of prediction and test scaled and unscaled arrays
print("First 5 entries of y_pred_scaled", y_pred_scaled[:5])  # Display first 5 rows
print("First 5 entries of y_pred_original", y_pred_original[:5])  # Display first 5 rows
print("First 5 entries of y_test_scaled", y_test_scaled[:5])  # Display first 5 target values
print("First 5 entries of y_test_original", y_test_original[:5])  # Display first 5 target values

# Mean Squared Error
mse = mean_squared_error(y_test_original, y_pred_original)
# Root Mean Squared Error
rmse = np.sqrt(mse)
# Mean Absolute Error
mae = mean_absolute_error(y_test_original, y_pred_original)
# R-squared
r2 = r2_score(y_test_original, y_pred_original)

# Print the metrics
print(f"Mean Squared Error (MSE): {mse:.6f}")
print(f"Root Mean Squared Error (RMSE): {rmse:.6f}")
print(f"Mean Absolute Error (MAE): {mae:.6f}")
print(f"R-squared (R²): {r2:.6f}")

First 5 entries of y_pred_scaled [[0.4498886 ]
 [0.42550546]
 [0.45606363]
 [0.43470904]
 [0.46578717]]
First 5 entries of y_pred_original [[ 0.0018089 ]
 [-0.00061966]
 [ 0.00242394]
 [ 0.00029702]
 [ 0.0033924 ]]
First 5 entries of y_test_scaled [[0.33935743]
 [0.50803213]
 [0.39056225]
 [0.4748996 ]
 [0.74096386]]
First 5 entries of y_test_original [[-0.0092]
 [ 0.0076]
 [-0.0041]
 [ 0.0043]
 [ 0.0308]]
Mean Squared Error (MSE): 0.000121
Root Mean Squared Error (RMSE): 0.010996
Mean Absolute Error (MAE): 0.008325
R-squared (R²): -0.022483


In [138]:
# Create a DataFrame to compare actual and predicted values
comparison_df = pd.DataFrame({'Actual': y_test_original.flatten(), 'Predicted': y_pred_original.flatten()})
# Calculate the difference (error)
comparison_df['Difference'] = comparison_df['Actual'] - comparison_df['Predicted']
#print(comparison_df.head(25))

In [139]:
# Get the sign of the actual and predicted values
actual_sign = np.sign(comparison_df['Actual'])
predicted_sign = np.sign(comparison_df['Predicted'])
#print(actual_sign.head(25))
#print(predicted_sign.head(25))
# Check where the signs match
sign_matches = actual_sign == predicted_sign
print(sign_matches.head(100))

# Calculate the percentage of sign matches
percentage_match = sign_matches.mean() * 100

print(f"Percentage of Sign Matches: {percentage_match :.2f}%")

0     False
1     False
2     False
3      True
4      True
      ...  
95    False
96     True
97    False
98     True
99    False
Length: 100, dtype: bool
Percentage of Sign Matches: 55.23%


In [140]:
# Get the prediction for tomorrow's Close_delta

"""
Predict the next Close_delta based on the most recent n_timesteps of data.
    
Using:
- model: The trained LSTM model.
- delta_only_df (pd.DataFrame): The DataFrame containing all historical data, including the latest Close_delta.
- n_timesteps (int): The number of timesteps (sequence length) used in the model.
- scaler_X: The scaler used to scale the features (X).
- scaler_y: The scaler used to scale the target (y).

Returns:
- float: The predicted Close_delta for the next day.
"""
# Step 1: Extract the last n_timesteps rows from the data (to be used as input for prediction)
last_window = delta_only_df[-n_timesteps:].values.reshape(1, n_timesteps, delta_only_df.shape[1])

# Step 2: Scale the input features
last_window_scaled = scaler_X.transform(last_window.reshape(-1, last_window.shape[-1])).reshape(1, n_timesteps, -1)

# Step 3: Use the model to predict the next Close_delta (scaled)
predicted_close_delta_scaled = model.predict(last_window_scaled)

# Step 4: Inverse transform the prediction to get the original scale of Close_delta
predicted_close_delta = scaler_y.inverse_transform(predicted_close_delta_scaled)

predicted_close_delta = predicted_close_delta[0][0]  # Return the predicted value

# Decision logic: Buy if positive, Sell if negative
if predicted_close_delta > 0:
    print(f"Predicted Close increase for {ticker} tomorrow is {(predicted_close_delta * 100):.2f}%. Suggestion: BUY with {percentage_match :.2f}% confidence")
else:
    print(f"Predicted Close decrease for {ticker} tomorrow is {(predicted_close_delta * 100):.2f}%. Suggestion: SELL with {percentage_match :.2f}% confidence")


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 74ms/step
Predicted Close increase for tomorrow is 0.30%. Suggestion: BUY with 55.23% confidence
